# Tweets - Topic Modeling with LDA

This is created following this [notebook](https://github.com/kapadias/mediumposts/blob/master/natural_language_processing/topic_modeling/notebooks/Introduction%20to%20Topic%20Modeling.ipynb)

In [17]:
# Importing modules
import pandas as pd
import os
from collections import defaultdict

tweets_per_peak = defaultdict(list)
t_count = 0
with open('../input/topicmodelling/all_tweets_neg_emo_words_with_emoji.tsv', mode='r', encoding='utf8') as fin:
    for line in fin:
        parts = line.strip().split('\t')
        
        if len(parts) > 2:
            tweets_per_peak[parts[1]].append(parts[2])
            t_count += 1

print(f'Total tweets: {t_count}, for {len(tweets_per_peak)} peak months')

Total tweets: 20086, for 26 peak months


## Processing

In [21]:
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import emoji
from tqdm import tqdm

stop_words = stopwords.words('english')
all_words = []
peaks = []

for d_key, d_val in tqdm(tweets_per_peak.items()):
    peaks.append(d_key)
    doc = ' '.join(d_val)
    doc = emoji.replace_emoji(doc, replace=' ')
    doc = re.sub(r'[^A-Za-z0-9]+', ' ', doc)
    doc = doc.lower()
    words = gensim.utils.simple_preprocess(doc, deacc=True)
    words = [w for w in words if w not in stop_words]
    all_words.append(words)

print(f'Number of docs: {len(all_words)}')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 26/26 [00:02<00:00, 11.10it/s]

Number of docs: 26


In [22]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(all_words)

# Create Corpus
texts = all_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 5), (4, 1), (5, 1), (6, 4), (7, 1), (8, 2), (9, 1), (10, 1), (11, 4), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 8), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [23]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"gay" + 0.011*"pride" + 0.007*"conversion" + 0.007*"rt" + '
  '0.006*"bullying" + 0.006*"people" + 0.006*"therapy" + 0.005*"news" + '
  '0.004*"homophobia" + 0.004*"sex"'),
 (1,
  '0.022*"gay" + 0.009*"pride" + 0.007*"conversion" + 0.007*"ban" + '
  '0.006*"therapy" + 0.005*"people" + 0.005*"new" + 0.005*"discrimination" + '
  '0.004*"rt" + 0.004*"porn"'),
 (2,
  '0.032*"gay" + 0.018*"pride" + 0.008*"conversion" + 0.007*"rt" + '
  '0.007*"people" + 0.006*"therapy" + 0.006*"ban" + 0.005*"news" + 0.005*"new" '
  '+ 0.005*"women"'),
 (3,
  '0.028*"gay" + 0.011*"pride" + 0.009*"rt" + 0.007*"therapy" + '
  '0.007*"conversion" + 0.005*"news" + 0.005*"bullying" + 0.005*"people" + '
  '0.004*"ban" + 0.004*"discrimination"'),
 (4,
  '0.035*"gay" + 0.018*"pride" + 0.011*"conversion" + 0.011*"therapy" + '
  '0.008*"rt" + 0.008*"ban" + 0.005*"news" + 0.005*"bullying" + 0.004*"new" + '
  '0.004*"people"'),
 (5,
  '0.016*"gay" + 0.008*"pride" + 0.007*"therapy" + 0.007*"conversion" + '


In [24]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')

LDAvis_prepared

/opt/conda/lib/python3.7/site-packages/past/types/oldstr.py:36: DeprecationWarning: invalid escape sequence \d
  """
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.007387 -0.004425       1        1  27.212754
2     -0.004006 -0.006141       2        1  21.451674
9     -0.010863  0.010422       3        1  17.115312
8      0.005589 -0.002055       4        1  13.108303
4     -0.011503 -0.001041       5        1  12.029362
3     -0.000617 -0.002147       6        1   8.656163
5      0.015372  0.003485       7        1   0.177867
0      0.005670 -0.004919       8        1   0.138208
1      0.006920  0.003740       9        1   0.095954
6      0.000825  0.003080      10        1   0.014403, topic_info=             Term         Freq        Total Category  logprob  loglift
504           gay  6069.000000  6069.000000  Default  30.0000  30.0000
980         pride  2879.000000  2879.000000  Default  29.0000  29.0000
1668   conversion  1973.000000  1973.000000  Default  28.0000  28.0000
2561      therapy  1712.000000  1712.000000  Default  27.0000  27.0000
1091           rt  1284.000000  1284.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
812         month     0.065258   606.054828  Topic10  -6.0665  -0.2909
930        people     0.083028   972.551486  Topic10  -5.8256  -0.5230
1193         stop     0.053934   437.093386  Topic10  -6.2571  -0.1546
967          porn     0.050878   472.228931  Topic10  -6.3154  -0.2903
1275  transgender     0.050187   450.536629  Topic10  -6.3291  -0.2569

[875 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
10375      1  0.278556     aba
10375      2  0.278556     aba
10375      3  0.278556     aba
1399       1  0.263637     abc
1399       2  0.160474     abc
...      ...       ...     ...
5323       2  0.230478  zagreb
5323       3  0.115239  zagreb
5323       4  0.230478  zagreb
5323       5  0.115239  zagreb
5323       6  0.115239  zagreb

[2461 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 10, 9, 5, 4, 6, 1, 2, 7])

/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
